<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Generate Leads from likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #post #comments #naas_drivers

## Input

### Import library

In [3]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [1]:
LI_AT = ''  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = ''  # EXAMPLE ajax:8379907400220387585

### Enter post URL

In [2]:
PROFILE_URL = "PROFILE_URL"

## Model

Get the information return in a dataframe.<br><br>
**Available columns :**
- PROFILE_URN : LinkedIn unique profile id
- PROFILE_ID : LinkedIn public profile id
- FIRSTNAME
- LASTNAME
- TEXT
- OCCUPATION
- ACTIVITY_COMMENTS
- ACTIVITY_LIKES
- DISTANCE
- POST_URL

In [4]:
DF_profile_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_stats(PROFILE_URL)

### Apply necessary filters to initial dataframe

- Get all post from profile
- Filter post with any of the empty value and add to new dataframe


In [6]:
DF_all_post_likes = pd.DataFrame()
for index, row in DF_profile_posts.iterrows():
    df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(row['POST_URL'])
    if not df.empty:
        DF_all_post_likes = DF_all_post_likes.append(df)

- Check the links with <b>PROFILE_ID</b> for users posts. (i.e. Here activity posts are filtered)
- Ignore below line if you want to consider all links

In [8]:
profile_posts = DF_all_post_likes[DF_all_post_likes['POST_URL'].str.contains('PROFILE_ID')]

In [9]:
grouped_profile_posts = profile_posts.groupby(['PROFILE_ID']).size().sort_values(ascending=False).reset_index(name='count')

- Group like count and classify the leads as per levels i.e. <b>L1, L2, L3, L4.</b> ( from highest to lowest counts of likes) with column name <b>"POTENTIAL LEAD"</b>
- Group posts as per the the column <b>"POTENTIAL LEAD"</b>

In [10]:
conditions = [
    (grouped_profile_posts['count'] <= 2),
    (grouped_profile_posts['count'] > 2) & (grouped_profile_posts['count'] <= 9),
    (grouped_profile_posts['count'] > 9) & (grouped_profile_posts['count'] <= 15),
    (grouped_profile_posts['count'] > 15)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [11]:
grouped_profile_posts['POTENTIAL_LEAD'] = np.select(conditions, values)

### Get profile details based on the POST_URL above
- Extract EMAIL, FIRSTNAME & LASTNAME from outputs of below cell

In [15]:
PROFILE_CONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(PROFILE_URL)
PROFILE_IDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(PROFILE_URL)

In [17]:
profile_Email = PROFILE_CONTACTS.at[0,'EMAIL']

In [18]:
profile_Name = PROFILE_IDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILE_IDENTITY.at[0,'LASTNAME']

- filter output based on L1, L2, & L3 levels and get profile details of potential leads and add to a list

In [19]:
leads_list = [];

r_count = 1
for index, row in grouped_profile_posts.iterrows():
    if row['POTENTIAL_LEAD'] == "L3" or row['POTENTIAL_LEAD'] == "L2" or row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_ID']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
       
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilephoneno = PROFILE_CONTACTS.at[0,'PHONENUMBER']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        profilefirstname = PROFILEIDENTITY.at[0,'FIRSTNAME']
        profilelastname = PROFILEIDENTITY.at[0,'LASTNAME']
        profileoccupation = PROFILEIDENTITY.at[0,'OCCUPATION']
        if profileemail != None :
            leads_list.append([r_count,profilename,profilefirstname,profilelastname,profileemail,profilephoneno,profileoccupation,profileurl])
            r_count = r_count + 1
   

### Create contacts from linkedin likes

In [29]:
HS_CONTACT_ASSIGN_TO_ID = "" #id to whom task needs to be assigned
HS_API_TOKEN = ""

In [30]:
HUB_COMPANY_ID = ""

In [31]:
HUB_CONTACT_URL = "https://app.hubspot.com/contacts/"+HUB_COMPANY_ID+"/contact/"

In [32]:
huboutput = ""

In [25]:
HS_CONTACT_ID_LIST = []
for i in leads_list:
        profilename = i[1]
        profilefirstname = i[2]
        profilelastname = i[3]
        profileemail = i[4]
        profilephoneno = i[5]
        profileoccupation = i[6]
        profileurl = i[7]
        # With send method
        data = {"properties": 
                {
                    "linkedinbio": profileurl,
                    "firstname": profilefirstname,
                    "lastname": profilelastname,
                    "jobtitle": profileoccupation,
                    "email": profileemail,
                    "phone": profilephoneno,
                }
               }
        #print(data)
        huboutput = hubspot.connect(HS_API_TOKEN).contacts.send(data)
        HS_CONTACT_ID_LIST.append([huboutput,profileurl])

✔️ Contact (id=4051) successfully created.
✔️ Contact (id=4101) successfully created.
✔️ Contact (id=4151) successfully created.
✔️ Contact (id=4201) successfully created.
✔️ Contact (id=4251) successfully created.
✔️ Contact (id=4301) successfully created.
✔️ Contact (id=4351) successfully created.
✔️ Contact (id=4352) successfully created.
✔️ Contact (id=4152) successfully created.
✔️ Contact (id=4401) successfully created.


## Output

### Display result - Create tasks for user and send email notification

In [33]:
url = "https://api.hubapi.com/engagements/v1/engagements"

querystring = { "hapikey": HS_API_TOKEN }

tstampobj = datetime.now() + timedelta(days=10)
tstamp = tstampobj.timestamp() * 1000

In [36]:
for i in HS_CONTACT_ID_LIST:
    hs = hubspot.connect(HS_API_TOKEN)
    contact_id = i[0]
    contact = hs.contacts.get(contact_id)
    contact_props = contact.get('properties')
    contact_linkedin_url = i[1]
    hs_contact_url = HUB_CONTACT_URL + contact_id
    payload = json.dumps({
        "engagement": {
            "active": 'true',
            "ownerId": "",
            "type": "TASK",
            "timestamp": tstamp
        },
        "associations": {
            "contactIds": [HS_CONTACT_ASSIGN_TO_ID],
            "companyIds": [],
            "dealIds": [],
            "ownerIds": [],
        },

        "metadata": {
            "body": "Hi there, you need to contact following user & task is already assigned to you.<br/>" + "Name :" +contact_props['firstname']+ contact_props['lastname'] + " Contact URL : " + hs_contact_url,
            "subject": "Task created for Contact ID :"+ contact_id,
            "status": "NOT_STARTED",
            "forObjectType": "CONTACT"
        }
    });
    headers = {
        'Content-Type': "application/json",
    }
    response = requests.request("POST", url, data=payload, headers=headers, params=querystring)

    if response.status_code == 200:
        print("Task created successfully for Contact ID :",contact_id)
        email_to = "" #to send the report
        subject = "LinkedIN Leads Alert"
        content = f"<p>Hi there,</p><br/><p>A new task has been created for you with contact ID :" + contact_id + "</p><br/><p>Actions : <a href="+hs_contact_url+">Open Hubspot Contact</a> | <a href="+contact_linkedin_url+">Open LinkedIN Profile</a></p>"

        naas.notification.send(email_to=email_to, subject=subject, html=content)
    else:
        print("Could not create the task for Contact ID :"+contact_id+", ask for on hello@naas.ai for more details")

Task created successfully for Contact ID : 4051
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4101
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4151
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4201
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4251
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4301
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4351
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4352
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4152
👌 💌 Email has been sent successfully !
Task created successfully for Contact ID : 4401
👌 💌 Email has been sent successfully !
